In [ ]:
import json as json
import matplotlib.pyplot as plt
import os
import numpy as np

import multifil
import treatment_definitions as td
import graphing_utilities as gutil

# List valid things that can be changed
print("valid constant (phenotype) parameters\n", multifil.hs.hs.VALID_PARAMS)

In [ ]:
# Setup experiment, show the traces
time_trace, length_trace, actin_permissiveness_trace, params = td.stiff_torsional_spring()

#plot the traces
gutil.plot_input_traces(time_trace, length_trace, actin_permissiveness_trace)

#print the phenotype settings


In [ ]:
"""This code runs a single simulation"""
output_dir = "../_originalTransitionRates/"

stiffness = 40 # torsional spring
params = {"mh_c_ks":stiffness, "mh_c_kw":stiffness, "detachment_rate":"force_dependent"}

# check the comment to see what we are passing to it.
wl_meta = multifil.aws.metas.emit(output_dir, # local dir to save run output to
                                  None, #s3 bucket to upload results to
                                  time_trace, 
                                  0.5, # poisson ratio, set to const vol here
                                 None, # intial ls, using default by passing none
                                 length_trace, 
                                 actin_permissiveness_trace, 
                                 str(stiffness), # comment describing run
                                 True, # whether to write out resulting file 
                                  #or just pass back to variable
                                 params)
run_name = wl_meta['name']
print(run_name)
run_file = output_dir + run_name + ".meta.json"

for key in wl_meta.keys():
    print(key)

print(wl_meta['hs_params'])

run_manager = multifil.aws.run.manage(run_file, False) # have run.py create the sarc from meta
# Begin running
run_manager.run_and_save()   # with our new sarc, run it

print(run_name)
data_name = run_file.split('.meta')[0] + '.data.json'

with open(data_name, 'r') as file:
    data = json.load(file)

for key in data.keys():
    print(key, end=", ")

print()
print()
# print_constants(data['constants'])

key = "axial_force"
fs = 16

# recreate time trace in milliseconds
time_trace = data['timestep'].copy()
for i in range(len(time_trace)):
    time_trace[i] *= data['timestep_length']

# plot
fig, axes = plt.subplots(figsize=(16, 9))
axes.plot(time_trace, data[key])

title = "multifilament model of muscle contraction\n" + wl_meta['comment']
plt.title(title, fontsize=fs*1.5)
plt.xlabel("time (ms)", fontsize=fs)
plt.ylabel(key, fontsize=fs)

ax2 = plt.twinx()
ax2.plot(time_trace, data['actin_permissiveness'])

save = True # OI! make sure you know whats going on here in this statement
if save:
    plt.savefig(output_dir + str(run_name) + ".png")

In [ ]:
plt.plot(data['xb_trans_31'])